<div align="center">
<h1>Stage 3: Feature Engineering</a></h1>
by Hongnan Gao
<br>
</div>

<a id="top"></a>

<a id = '1.0'></a>
<h1 style = "font-family: garamond; font-size: 40px; font-style: normal;background-color: #2ab7ca; color : #fed766; border-radius: 5px 5px;padding:5px;text-align:center; font-weight: bold" >Quick Navigation</h1>

    
* [Dependencies and Configuration](#1)
* [Stage 3: Feature Engineering/Feature Selection](#2)
    * [Multicollinearity and Feature Selection](#31)
        * [Target Distribution](#31)
        * [Using Statsmodels Variance Inflation Factor](#31)
        * [Oh Dear, we have a Multicollinearity Problem](#31)
    * [Save the Data](#31)

## Dependencies and Configuration

In [1]:
import random
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Dict, List, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import (base, decomposition, linear_model, manifold, metrics,
                     preprocessing)
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
@dataclass
class config:
    raw_data: str = "https://storage.googleapis.com/reighns/reighns_ml_projects/docs/supervised_learning/classification/breast-cancer-wisconsin/data/raw/data.csv"
    processed_data: str = "https://storage.googleapis.com/reighns/reighns_ml_projects/docs/supervised_learning/classification/breast-cancer-wisconsin/data/processed/processed.csv"
    train_size: float = 0.9
    seed: int = 1992
    num_folds: int = 5
    cv_schema: str = "StratifiedKFold"
    classification_type: str = "binary"

    target_col: List[str] = field(default_factory=lambda: ["diagnosis"])
    unwanted_cols: List[str] = field(default_factory=lambda: ["id", "Unnamed: 32"])

    # Plotting
    colors: List[str] = field(
        default_factory=lambda: ["#fe4a49", "#2ab7ca", "#fed766", "#59981A"]
    )
    cmap_reversed = plt.cm.get_cmap("mako_r")

    def to_dict(self) -> Dict:
        """Convert the config object to a dictionary.

        Returns:
            Dict: The config object as a dictionary.
        """
        return {
            "raw_data": self.raw_data,
            "processed_data": self.processed_data,
            "train_size": self.train_size,
            "seed": self.seed,
            "num_folds": self.num_folds,
            "cv_schema": self.cv_schema,
            "classification_type": self.classification_type,
            "target_col": self.target_col,
            "unwanted_cols": self.unwanted_cols,
            "colors": self.colors,
            "cmap_reversed": self.cmap_reversed,
        }

In [3]:
def set_seeds(seed: int = 1234) -> None:
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)

In [4]:
config = config()

# set seeding for reproducibility
_ = set_seeds(seed = config.seed)

# read data
df = pd.read_csv(config.processed_data)

# Stage 3: Feature Engineering/Feature Selection

!!! danger "Foreward on Data Leakage"
    We are fully aware that oftentimes practitioner may accidentally cause data leakage during preprocessing, for example, a subtle yet often mistake is to standardize the whole dataset prior to splitting, or performing feature selection prior to modelling using the information of our response/target variable. 
    However, this does not mean we should not do any preprocessing before modelling, instead, in the case of removing multicollinearity features, we can still screen predictors for multicollinearity during EDA phase and have a good intuition on which predictors are highly correlated - subsequently, we will incorporate feature selection techniques in our modelling pipeline.


### Multicollinearity and Feature Selection

!!! note "Why Feature Selection?"
    We need feature selection in certain problems for the following reasons:
    - Well, one would definitely have heard of the dreaded **Curse of Dimensionality** in the journey of learning Machine Learning where having too many predictor/features can lead to overfitting; on the other hand, too many dimensions can cause distance between observations to appear equidistance from one another. The equidistance phenomenon causes observations to become harder to cluster, thereby clogging the model's ability to cluster data points (imagine the horror if you use KNN on 1000 dimensions, all the points will be almost the same distance from each other, poor KNN will not know how to predict now).
    - In case you have access to Google's GPU clusters, you likely want to train your model faster. Reducing the number predictors can aid this process.
    - Reducing uninformative features may aid in model's performance, the idea is to remove unnecessary noise from the dataset.

---

!!! danger "Multi-Collinearity"
    Looking back at our dataset, it is clear to me that there are quite a number of features that are correlated with each other, causing multi-collinearity. Multi-Collinearity is an issue in the history of Linear Models, as quoted[^Is there an intuitive explanation why multicollinearity is a problem in linear regression?]
    
    > Consider the simplest case where Y is regressed against X and Z and where X and Z are highly positively correlated. Then the effect of X on Y is hard to distinguish from the effect of Z on Y because any increase in X tends to be associated with an increase in Z.

    We also note that multi-collinearity is not that big of a problem for non-parametric models such as Decision Tree or Random Forests, however, I will attempt to show that it is still best to avoid in this problem setting.

[^Is there an intuitive explanation why multicollinearity is a problem in linear regression?]: https://stats.stackexchange.com/questions/1149/is-there-an-intuitive-explanation-why-multicollinearity-is-a-problem-in-linear-r

!!! alert "Feature Selection Methods"
    There are many methods to perform feature selection. Scikit-Learn offers some of the following:
    - Univariate feature selection.
    - Recursive feature elimination.
    - Backward Elimination of features using Hypothesis Testing.  
    We need to be careful when selecting features before cross-validation. It is therefore, recommended to include feature selection in cross-validation to avoid any "bias" introduced before model selection phase! I decided to use the good old Variance Inflation Factor (VIF) as a way to reduce multicollinearity. Unfortunately, there is no out-of-the-box function to integrate into the `Pipeline` of scikit-learn. Thus, I heavily modified an existing code in order achieve what I want below.


### Variance Inflation Factor

A classical way to check for multicollinearity amongst predictors is to calculate the Variable Inflation Factor (VIF). It is simply done by regressing each predictor $\mathrm{x}_i$ against all other predictors $\mathrm{x}_j, j \neq i$. In other words, the VIF for a predictor variable $i$ is given by:

$$\text{VIF}_i = \dfrac{1}{1 - R^{2}_{i}}$$

where $R^{2}_{i}$ is, by definition, the proportion of the variation in the "dependent variable" $\mathrm{x}_i$ that is predictable from the indepedent predictors $\mathrm{x}_j, j \neq i$. Consequently, the higher the $R^2_i$ of a predictor, the higher the VIF, and this indicates there is linear dependence among predictors.

#### Using Statsmodels Variance Inflation Factor

Note that we need to perform scaling first before fitting our `ReduceVIF` to get the exact same result as the previous version. In this version, I manually added a hard threshold for the number of features remaining to be 15. This hard coded number can be turned into a parameter (hyperparameter) in our pipeline.

In [5]:
import numpy as np
import pandas as pd
from sklearn import base
from statsmodels.regression.linear_model import OLS

def variance_inflation_factor(exog, idx_kept, vif_idx):
    """Compute VIF for one feature.
    
    Args:
        exog (np.ndarray): Observations
        idx_kept (List[int]): Indices of features to consider
        vif_idx (int): Index of feature for which to compute VIF
    
    Returns:
        float: VIF for the selected feature
    """
    exog = np.asarray(exog)
    
    x_i = exog[:, vif_idx]
    mask = [col for col in idx_kept if col != vif_idx]
    x_noti = exog[:, mask]
    
    r_squared_i = OLS(x_i, x_noti).fit().rsquared
    vif = 1. / (1. - r_squared_i)
    
    return vif

class ReduceVIF(base.BaseEstimator, base.TransformerMixin):
    """The base of the class structure is implemented in https://www.kaggle.com/ffisegydd/sklearn-multicollinearity-class;
    I heavily modified the class such that it can take in numpy arrays and correctly implemented the fit and transform method.
    """

    def __init__(self, thresh=10, max_drop=20):
        self.thresh = thresh
        self.max_drop = max_drop
        self.column_indices_kept_ = []
        self.feature_names_kept_ = None

    def reset(self):
        """Resets the state of predictor columns after each fold."""

        self.column_indices_kept_ = []
        self.feature_names_kept_ = None

    def fit(self, X, y=None):
        """Fits the Recursive VIF on the training folds and save the selected feature names in self.feature_names

        Args:
            X ([type]): [description]
            y ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """
        
        self.column_indices_kept_, self.feature_names_kept_ = self.calculate_vif(X)
        
        return self

    def transform(self, X, y=None):
        """Transforms the Validation Set according to the selected feature names.

        Args:
            X ([type]): [description]
            y ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """

        return X[:, self.column_indices_kept_]

    def calculate_vif(self, X: Union[np.ndarray, pd.DataFrame]):
        """Implements a VIF function that recursively eliminates features.

        Args:
            X (Union[np.ndarray, pd.DataFrame]): [description]

        Returns:
            [type]: [description]
        """
        feature_names = None
        column_indices_kept = list(range(X.shape[1]))
        
        if isinstance(X, pd.DataFrame):
            feature_names = X.columns

        dropped = True
        count = 0
        
        while dropped and count <= self.max_drop:
            dropped = False
            
            max_vif, max_vif_col = None, None
            
            for col in column_indices_kept:
                
                vif = variance_inflation_factor(X, column_indices_kept, col)
                
                if max_vif is None or vif > max_vif:
                    max_vif = vif
                    max_vif_col = col
            
            if max_vif > self.thresh:
                print(f"Droppingggggg {max_vif_col} with vif={max_vif}")
                column_indices_kept.remove(max_vif_col)
                
                if feature_names is not None:
                    feature_names.pop(max_vif_col)
                    
                dropped = True
                count += 1
                
        return column_indices_kept, feature_names


We do a sanity check if this coincides with the previous defined class, and the results are the same.

In [6]:
predictor_cols = df.columns[1:]
transformer = ReduceVIF()
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(df[predictor_cols])
# Only use 10 columns for speed in this example
X = transformer.fit_transform(X)

print(f"Remaining Features: {transformer.column_indices_kept_}")

Droppingggggg 0 with vif=3806.1152963979675
Droppingggggg 20 with vif=616.3508614719424
Droppingggggg 2 with vif=325.64131198187516
Droppingggggg 22 with vif=123.25781086343038
Droppingggggg 6 with vif=64.65479584770004
Droppingggggg 10 with vif=35.61751844352034
Droppingggggg 25 with vif=33.96063880508537
Droppingggggg 27 with vif=30.596655364834078
Droppingggggg 3 with vif=25.387829695531458
Droppingggggg 5 with vif=18.843208489973282
Droppingggggg 21 with vif=17.232376192128665
Droppingggggg 13 with vif=16.333806476471736
Droppingggggg 26 with vif=15.510661467365699
Remaining Features: [1, 4, 7, 8, 9, 11, 12, 14, 15, 16, 17, 18, 19, 23, 24, 28, 29]


We have the remaining indices, and therefore simply use numpy to subset the column indices to get back the original column names that are kept.

In [7]:
vif_df = pd.DataFrame({'Predictors': predictor_cols[transformer.column_indices_kept_]})

In [8]:
display(vif_df)

,Predictors
0,texture_mean
1,smoothness_mean
2,concave points_mean
3,symmetry_mean
4,fractal_dimension_mean
5,texture_se
6,perimeter_se
7,smoothness_se
8,compactness_se
9,concavity_se


#### Oh Dear, we have a Multicollinearity Problem

!!! info "Using VIF in Modelling Pipeline"
    At this step, we are just showing how we can remove multicollinear features using VIF; but we will not remove them at this point in time. We will incorporate this feature selection technique in our Cross-Validation pipeline in order to avoid data leakage.


## Save the Data

In this phase, we did not make any changes to the predictor or target columns. So there is nothing to save.